# Metaphor Label Projection: ES/EN → CA

Based on **Meta4XNLI** methodology (Sanchez-Bayona & Agerri, 2025)

This notebook projects metaphor labels from Spanish and English to Catalan using:
- Word alignment (multilingual BERT embeddings)
- BIO tag consistency enforcement
- The projection methodology described in Section 3.2.1 of the paper

## 1. Setup and Installation

In [ ]:
# Install required packages (run once)
!pip install transformers torch sentencepiece -q

import os
from pathlib import Path
from typing import List, Tuple, Dict
from collections import defaultdict
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm

print("✓ Setup complete")

## 2. Mount Google Drive (if using Colab)

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = '/content/drive/MyDrive/HLE-project/data/meta4xnli/detection'  # Adjust to your data location
except:
    BASE_PATH = './'  # Local directory
    print("Running locally")

## 3. Configuration

In [ ]:
# Adjust these paths to match your file structure
SOURCE_DIR = os.path.join(BASE_PATH, "source_datasets_filtered")
OUTPUT_DIR = os.path.join(BASE_PATH, "projected_labels")

# Files to process
FILES = [
    'xnli_dev_hyp.tsv',
    'xnli_dev_prem.tsv',
    'xnli_test_hyp.tsv',
    'xnli_test_prem.tsv'
]

print(f"Source directory: {SOURCE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Files to process: {FILES}")

## 4. Load Alignment Model

We use multilingual BERT for computing word alignments, following the paper's approach.

In [ ]:
print("Loading multilingual BERT model...")
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')
model.eval()
print("✓ Model loaded")

## 5. Helper Functions

In [ ]:
def read_tsv_file(filepath: str) -> List[Tuple[List[str], List[str]]]:
    """Read CoNLL-format TSV file."""
    sentences = []
    current_tokens, current_labels = [], []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if current_tokens:
                    sentences.append((current_tokens, current_labels))
                    current_tokens, current_labels = [], []
            else:
                parts = line.split('\t')
                if len(parts) >= 2:
                    current_tokens.append(parts[0])
                    current_labels.append(parts[1])
    
    if current_tokens:
        sentences.append((current_tokens, current_labels))
    
    return sentences

def write_tsv_file(filepath: str, sentences: List[Tuple[List[str], List[str]]]):
    """Write CoNLL-format TSV file."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for tokens, labels in sentences:
            for token, label in zip(tokens, labels):
                f.write(f"{token}\t{label}\n")
            f.write("\n")

def align_tokens(src_tokens: List[str], tgt_tokens: List[str], 
                tokenizer, model, threshold: float = 0.7) -> Dict[int, List[int]]:
    """Align tokens using multilingual BERT embeddings."""
    src_text = " ".join(src_tokens)
    tgt_text = " ".join(tgt_tokens)
    
    # Tokenize
    src_encoded = tokenizer(src_text, return_tensors='pt', add_special_tokens=True)
    tgt_encoded = tokenizer(tgt_text, return_tensors='pt', add_special_tokens=True)
    
    # Get embeddings
    with torch.no_grad():
        src_embeddings = model(**src_encoded).last_hidden_state[0]
        tgt_embeddings = model(**tgt_encoded).last_hidden_state[0]
    
    # Compute similarity
    similarity = torch.matmul(src_embeddings, tgt_embeddings.T)
    
    # Map subwords to words
    src_word_ids = src_encoded.word_ids()
    tgt_word_ids = tgt_encoded.word_ids()
    
    # Build alignments
    alignments = defaultdict(list)
    
    for src_idx in range(len(src_tokens)):
        src_sub = [i for i, wid in enumerate(src_word_ids) if wid == src_idx]
        if not src_sub:
            continue
            
        for tgt_idx in range(len(tgt_tokens)):
            tgt_sub = [i for i, wid in enumerate(tgt_word_ids) if wid == tgt_idx]
            if not tgt_sub:
                continue
            
            max_sim = max(similarity[s, t].item() for s in src_sub for t in tgt_sub)
            
            if max_sim > threshold:
                if tgt_idx not in alignments[src_idx]:
                    alignments[src_idx].append(tgt_idx)
    
    return dict(alignments)

def fix_bio_consistency(labels: List[str]) -> List[str]:
    """Fix BIO tag consistency."""
    fixed = []
    prev = 'O'
    
    for label in labels:
        if label == 'I-METAPHOR' and prev not in ['B-METAPHOR', 'I-METAPHOR']:
            fixed.append('B-METAPHOR')
        else:
            fixed.append(label)
        prev = fixed[-1]
    
    return fixed

def project_labels(src_tokens: List[str], src_labels: List[str],
                  tgt_tokens: List[str], tokenizer, model) -> List[str]:
    """Project labels from source to target."""
    tgt_labels = ['O'] * len(tgt_tokens)
    
    # Get alignments
    alignments = align_tokens(src_tokens, tgt_tokens, tokenizer, model)
    
    # Project metaphor labels
    for src_idx, src_label in enumerate(src_labels):
        if src_label != 'O' and src_idx in alignments:
            for tgt_idx in alignments[src_idx]:
                if tgt_idx < len(tgt_labels):
                    tgt_labels[tgt_idx] = src_label
    
    # Fix BIO consistency
    tgt_labels = fix_bio_consistency(tgt_labels)
    
    return tgt_labels

print("✓ Helper functions defined")

## 6. Projection Function

In [ ]:
def project_file(src_file: str, tgt_file: str, output_file: str, 
                tokenizer, model, lang_pair: str):
    """Project labels from source to target file."""
    print(f"\n{'='*60}")
    print(f"Projecting: {os.path.basename(src_file)}")
    print(f"Language pair: {lang_pair}")
    print(f"{'='*60}")
    
    # Read files
    src_sentences = read_tsv_file(src_file)
    tgt_sentences = read_tsv_file(tgt_file)
    
    if len(src_sentences) != len(tgt_sentences):
        print(f"⚠ WARNING: Sentence count mismatch!")
        print(f"  Source: {len(src_sentences)}, Target: {len(tgt_sentences)}")
    
    # Project
    projected = []
    metaphor_count = 0
    
    for (src_tok, src_lab), (tgt_tok, _) in tqdm(zip(src_sentences, tgt_sentences), 
                                                   total=len(src_sentences),
                                                   desc="Projecting"):
        proj_lab = project_labels(src_tok, src_lab, tgt_tok, tokenizer, model)
        projected.append((tgt_tok, proj_lab))
        metaphor_count += sum(1 for l in proj_lab if l != 'O')
    
    # Write output
    write_tsv_file(output_file, projected)
    
    print(f"\n✓ Complete!")
    print(f"  Projected {metaphor_count} metaphor tokens")
    print(f"  Output: {output_file}")
    
    return metaphor_count

print("✓ Projection function ready")

## 7. Run Projection: Spanish → Catalan

In [ ]:
print("\n" + "="*70)
print("SPANISH → CATALAN PROJECTION")
print("="*70)

es_ca_stats = {}

for filename in FILES:
    src_file = os.path.join(SOURCE_DIR, 'es', filename)
    tgt_file = os.path.join(SOURCE_DIR, 'ca', filename)
    output_file = os.path.join(OUTPUT_DIR, 'ca-es', filename)
    
    if os.path.exists(src_file) and os.path.exists(tgt_file):
        count = project_file(src_file, tgt_file, output_file, 
                           tokenizer, model, "ES → CA")
        es_ca_stats[filename] = count
    else:
        print(f"\n⚠ Skipping {filename} - files not found")

print(f"\n{'='*70}")
print("ES → CA SUMMARY")
print(f"{'='*70}")
for fname, count in es_ca_stats.items():
    print(f"  {fname}: {count} metaphor tokens")

## 8. Run Projection: English → Catalan

In [ ]:
print("\n" + "="*70)
print("ENGLISH → CATALAN PROJECTION")
print("="*70)

en_ca_stats = {}

for filename in FILES:
    src_file = os.path.join(SOURCE_DIR, 'en', filename)
    tgt_file = os.path.join(SOURCE_DIR, 'ca', filename)
    output_file = os.path.join(OUTPUT_DIR, 'ca-en', filename)
    
    if os.path.exists(src_file) and os.path.exists(tgt_file):
        count = project_file(src_file, tgt_file, output_file, 
                           tokenizer, model, "EN → CA")
        en_ca_stats[filename] = count
    else:
        print(f"\n⚠ Skipping {filename} - files not found")

print(f"\n{'='*70}")
print("EN → CA SUMMARY")
print(f"{'='*70}")
for fname, count in en_ca_stats.items():
    print(f"  {fname}: {count} metaphor tokens")

## 9. Final Summary and Next Steps

In [ ]:
print("\n" + "="*70)
print("PROJECTION COMPLETE!")
print("="*70)

print("\nOutput directories created:")
print(f"  • {os.path.join(OUTPUT_DIR, 'ca-es')} (Spanish → Catalan)")
print(f"  • {os.path.join(OUTPUT_DIR, 'ca-en')} (English → Catalan)")

print("\n📊 Statistics:")
print(f"\nSpanish → Catalan:")
for fname, count in es_ca_stats.items():
    print(f"  {fname}: {count} tokens")
    
print(f"\nEnglish → Catalan:")
for fname, count in en_ca_stats.items():
    print(f"  {fname}: {count} tokens")

print("\n⚠️  Next Steps (from Meta4XNLI paper):")
print("  1. Manual review of projected labels")
print("  2. Correction of alignment errors")
print("  3. Addition of missed metaphors")
print("  4. Verification of BIO tag consistency")
print("  5. Inter-annotator agreement check (recommended)")

print("\n💡 Key Findings from the Paper:")
print("  • Not all metaphors transfer across languages")
print("  • Translation affects metaphor preservation (see Table 9)")
print("  • Manual revision improved quality significantly")
print("  • Semi-automatic approach is efficient but needs validation")